# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.

%load_ext dotenv
%dotenv ../01_materials/labs/.env

In [2]:
import dask.dataframe as dd

/var/folders/93/fw4w66pd2sgcwbnxdlq9215c0000gn/T/ipykernel_35077/676544213.py:1: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask DataFrame will use new implementation that
contains several improvements including a logical query planning.
The user-facing DataFrame API will remain unchanged.

The new implementation is already available and can be enabled by
installing the dask-expr library:

    $ pip install dask-expr

and turning the query planning option on:

    >>> import dask
    >>> dask.config.set({'dataframe.query-planning': True})
    >>> import dask.dataframe as dd

API documentation for the new implementation is available at
https://docs.dask.org/en/stable/dask-expr-api.html

Any feedback can be reported on the Dask issue tracker
https://github.com/dask/dask/issues 

  import dask.dataframe as dd


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [28]:
import os
from glob import glob

# Write your code below.
#PRICE_DATA = os.getenv ("PRICE_DATA")
#glob(os.path.join(PRICE_DATA, "**/part.0.parquet"), recursive = True)

PRICE_DATA = "/Users/Jakub_1/DSI/production/02_activities/assignments/../../05_src/data/prices/"
parquet_files = glob(os.path.join(PRICE_DATA, "**/part.0.parquet"), recursive=True)
parquet_files




['/Users/Jakub_1/DSI/production/02_activities/assignments/../../05_src/data/prices/CTAS/CTAS_2008.parquet/part.0.parquet',
 '/Users/Jakub_1/DSI/production/02_activities/assignments/../../05_src/data/prices/CTAS/CTAS_2018.parquet/part.0.parquet',
 '/Users/Jakub_1/DSI/production/02_activities/assignments/../../05_src/data/prices/CTAS/CTAS_2011.parquet/part.0.parquet',
 '/Users/Jakub_1/DSI/production/02_activities/assignments/../../05_src/data/prices/CTAS/CTAS_2001.parquet/part.0.parquet',
 '/Users/Jakub_1/DSI/production/02_activities/assignments/../../05_src/data/prices/CTAS/CTAS_2000.parquet/part.0.parquet',
 '/Users/Jakub_1/DSI/production/02_activities/assignments/../../05_src/data/prices/CTAS/CTAS_2010.parquet/part.0.parquet',
 '/Users/Jakub_1/DSI/production/02_activities/assignments/../../05_src/data/prices/CTAS/CTAS_2019.parquet/part.0.parquet',
 '/Users/Jakub_1/DSI/production/02_activities/assignments/../../05_src/data/prices/CTAS/CTAS_2009.parquet/part.0.parquet',
 '/Users/Jakub_1

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [31]:
# Write your code below.

import dask.dataframe as dd
import numpy as np 

# loading parquet files into dask dataframe
ddf = dd.read_parquet(parquet_files)

dd_feat = (ddf.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag = x['Close'].shift(1), Adj_Close_lag = x['Adj Close'].shift(1))
).assign(
    returns = lambda x: x['Adj Close']/x['Adj_Close_lag'] - 1
).assign(
    hi_lo_range = lambda x: (x['High'] - x['Low']
)))
dd_feat.compute()

/var/folders/93/fw4w66pd2sgcwbnxdlq9215c0000gn/T/ipykernel_35077/600235154.py:9: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (ddf.groupby('ticker', group_keys=False).apply(


,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag,Adj_Close_lag,returns,hi_lo_range
ticker,,,,,,,,,,,,,,
HUM,2001-01-02,14.500000,15.250000,14.312500,14.937500,13.337628,933100,Health Care,Managed Health Care,2001,NaN,NaN,NaN,0.937500
HUM,2001-01-03,14.687500,15.625000,14.250000,14.437500,12.891180,1077200,Health Care,Managed Health Care,2001,14.937500,13.337628,-0.033473,1.375000
HUM,2001-01-04,14.375000,14.437500,12.250000,13.187500,11.775059,1217600,Health Care,Managed Health Care,2001,14.437500,12.891180,-0.086580,2.187500
HUM,2001-01-05,13.000000,14.250000,12.750000,13.625000,12.165698,1606100,Health Care,Managed Health Care,2001,13.187500,11.775059,0.033175,1.500000
HUM,2001-01-08,13.625000,14.500000,13.312500,13.562500,12.109898,873700,Health Care,Managed Health Care,2001,13.625000,12.165698,-0.004587,1.187500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JNPR,2019-12-24,24.510000,24.510000,24.270000,24.290001,21.141632,714900,Information Technology,Communications Equipment,2019,24.469999,21.298300,-0.007356,0.240000
JNPR,2019-12-26,24.290001,24.530001,24.290001,24.490000,21.315706,1141000,Information Technology,Communications Equipment,2019,24.290001,21.141632,0.008234,0.240000
JNPR,2019-12-27,24.570000,24.610001,24.389999,24.410000,21.246075,3030300,Information Technology,Communications Equipment,2019,24.490000,21.315706,-0.003267,0.220001


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [32]:
# Write your code below.

pdf = dd_feat.compute()
pdf['rolling average return'] = pdf['returns'].rolling(10).mean()
pdf.head()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag,Adj_Close_lag,returns,hi_lo_range,rolling average return
ticker,,,,,,,,,,,,,,,
HUM,2001-01-02,14.5000,15.2500,14.3125,14.9375,13.337628,933100,Health Care,Managed Health Care,2001,NaN,NaN,NaN,0.9375,NaN
HUM,2001-01-03,14.6875,15.6250,14.2500,14.4375,12.891180,1077200,Health Care,Managed Health Care,2001,14.9375,13.337628,-0.033473,1.3750,NaN
HUM,2001-01-04,14.3750,14.4375,12.2500,13.1875,11.775059,1217600,Health Care,Managed Health Care,2001,14.4375,12.891180,-0.086580,2.1875,NaN
HUM,2001-01-05,13.0000,14.2500,12.7500,13.6250,12.165698,1606100,Health Care,Managed Health Care,2001,13.1875,11.775059,0.033175,1.5000,NaN
HUM,2001-01-08,13.6250,14.5000,13.3125,13.5625,12.109898,873700,Health Care,Managed Health Care,2001,13.6250,12.165698,-0.004587,1.1875,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
#no, dask can also do rolling averages.
+ Would it have been better to do it in Dask? Why?
#can be better if we're working with a lot of data since dask does the computations out of the core

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.